In [1]:
import numpy as np 
import os
import cv2



In [2]:
dir_path = "../DataSet" 
actions = np.array(os.listdir(dir_path))



In [4]:
test_actions = ['Blue','Happy','Man', 'Family']

max_frames_per_video = 0 
max_height = 0 
max_width = 0 
channels = 3 

for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        max_frames_per_video = max(max_frames_per_video, num_of_frames)
        max_height = max(max_height, height)
        max_width = max(max_width, width)
        cap.release()

data = np.zeros((len(test_actions)*num_files, max_frames_per_video, max_height, max_width, channels), dtype = np.uint8)


In [5]:
for i, action in enumerate(test_actions): 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
    
    for j, file in enumerate(file_list): 
        file_name = file_list[j]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        frames = np.zeros((max_frames_per_video, max_height, max_width, channels), dtype=np.uint8)

        k = 0 
        while True: 
            ret, frame = cap.read()
            if not ret or k >= max_frames_per_video: 
                break
          
            if frame.shape[0] <= max_height and frame.shape[1] <= max_width: 
             frame = cv2.resize(frame,(max_width, max_height))
             frames [k, :frame.shape[0], :frame.shape[1], :] = frame
             print("in if")
            else: 
               continue
            k += 1
        
        cap.release()
      
        frames = frames[:k, :, :, :]
        data[i * num_files + j,  :k, :max_height, :max_width, :] = frames


dask_data = da.from_array(data, chunks=(1,max_frames_per_video, max_height, max_width, channels))



in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in if
in i

## formatting the data

In [3]:
#Pre processing data and creating labels 
from tensorflow.keras.utils import to_categorical


label_map = {label: num for num, label in enumerate(actions)}

In [33]:
print(label_map)

{'Blue': 0, 'Family': 1, 'Happy': 2, 'Man': 3}


In [4]:

labels = []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)

    for video in range(len(file_list)): 
        labels.append(label_map[action])


In [5]:
import pandas as pd
test_actions = ['Blue','Happy','Man', 'Family']

video_features = []
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()
        features = np.array(frames)

        video_features.append(features)

df = pd.DataFrame({'class': labels, 'features': video_features})


In [41]:
df.to_csv('video_features.csv')

In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)

In [7]:
def process_block(block): 
    np_block = block.compute()
    processed_block = np.mean(np_block, axis=0)

    return da.from_array(processed_block, dask_data, dtype= np.float32)


dask_data_processed = da.map_blocks(process_block, dask_data, dtype=np.float32)

In [8]:
import dask_ml.model_selection as dcv 
import tensorflow as tf 
X_train,X_test, y_train, y_test = dcv.train_test_split(dask_data_processed, labels_dask, test_size = 0.1 )
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.compute(), y_train.compute()))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.compute(), y_test.compute()))

MemoryError: Unable to allocate 200. GiB for an array with shape (120, 72, 1080, 1920, 3) and data type float32

## Training the model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, Dense , Masking, Flatten, MaxPooling3D

In [8]:
model = Sequential()
# model.add(Masking(mask_value=-999, input_shape =input_shape))
# model.add(Masking(mask_value=-999, input_shape =padded_sequences.shape))

model.add(Conv3D(16, kernel_size=(3,3,3), activation='relu',input_shape =train_df['features'].iloc[0].shape))
model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(Conv3D(32, kernel_size=(3,3,3), activation = 'relu'))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

ResourceExhaustedError: OOM when allocating tensor with shape[142021584,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 35, 958, 1278, 32) 2624      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 17, 479, 639, 32)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 477, 637, 64)  55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 238, 318, 64)   0         
_________________________________________________________________
flatten (Flatten)            (None, 33906432)          0         
_________________________________________________________________
dense (Dense)                (None, 128)               45056128  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

In [19]:
model.fit(train_df['features'].tolist(),train_df['filename'], epochs=2000)

ValueError: Data cardinality is ambiguous:
  x sizes: 37, 27, 38
  y sizes: 3
Make sure all arrays contain the same number of samples.